backward selection

In [1]:
#Some basic setups
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
#Replacement for ISLR2 library
import statsmodels.api as sm
# different approach with sklearn
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)


#We can try using get_dummies from panda library for binarization.

#Replacement for FNN library for KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

#Replacement for leaps library
from sklearn.feature_selection import SelectKBest, f_regression
from mlxtend.feature_selection import ExhaustiveFeatureSelector

#Replacement for caTools library
from sklearn.model_selection import train_test_split

#Replacement for glmnet library for CV
from sklearn.linear_model import ElasticNetCV

#Replacement for glmnet library for Ridge and Lasso
from sklearn.linear_model import ElasticNet

#Replacement for pls library
from sklearn.cross_decomposition import PLSRegression


In [3]:
df = pd.read_csv("/content/dataQTM.csv")
#M = 0, B = 1
y = df['diagnosis']
X = df.drop(['diagnosis', 'id'], axis=1).astype('float64')

# Define the feature set X.
Xcons = X
Xcons = sm.add_constant(Xcons)
print(Xcons)

     const  radius_0ean  texture_0ean  peri0eter_0ean  area_0ean  \
0      1.0        17.99         10.38          122.80     1001.0   
1      1.0        20.57         17.77          132.90     1326.0   
2      1.0        19.69         21.25          130.00     1203.0   
3      1.0        11.42         20.38           77.58      386.1   
4      1.0        20.29         14.34          135.10     1297.0   
..     ...          ...           ...             ...        ...   
564    1.0        21.56         22.39          142.00     1479.0   
565    1.0        20.13         28.25          131.20     1261.0   
566    1.0        16.60         28.08          108.30      858.1   
567    1.0        20.60         29.33          140.10     1265.0   
568    1.0         7.76         24.54           47.92      181.0   

     s0oothness_0ean  co0pactness_0ean  concavity_0ean  concave points_0ean  \
0            0.11840           0.27760         0.30010              0.14710   
1            0.08474     

In [13]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y,Xcons[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(Xcons[list(feature_set)]) - y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [14]:
import time
import itertools
def backward(predictors):

    tic = time.time()

    results = []

    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))

    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)

    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]

    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (toc-tic), "seconds.")

    # Return the best model, along with some other useful information about the model
    return best_model

In [15]:
models_bwd = pd.DataFrame(columns=["RSS", "model"], index = range(1,len(Xcons.columns)))

tic = time.time()
predictors = Xcons.columns

while(len(predictors) > 1):
    models_bwd.loc[len(predictors)-1] = backward(predictors)
    predictors = models_bwd.loc[len(predictors)-1]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  31 models on 30 predictors in 0.4084641933441162 seconds.
Processed  30 models on 29 predictors in 0.3981964588165283 seconds.
Processed  29 models on 28 predictors in 0.3923778533935547 seconds.
Processed  28 models on 27 predictors in 0.4234733581542969 seconds.
Processed  27 models on 26 predictors in 0.33770751953125 seconds.
Processed  26 models on 25 predictors in 0.3990347385406494 seconds.
Processed  25 models on 24 predictors in 0.29621124267578125 seconds.
Processed  24 models on 23 predictors in 0.29180049896240234 seconds.
Processed  23 models on 22 predictors in 0.2936999797821045 seconds.
Processed  22 models on 21 predictors in 0.2503492832183838 seconds.
Processed  21 models on 20 predictors in 0.2796788215637207 seconds.
Processed  20 models on 19 predictors in 0.3003532886505127 seconds.
Processed  19 models on 18 predictors in 0.29349279403686523 seconds.
Processed  18 models on 17 predictors in 0.24322271347045898 seconds.
Processed  17 models on 16 predi

In [16]:
print(models_bwd.loc[21, "model"].summary())

                            OLS Regression Results                            
Dep. Variable:              diagnosis   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                     93.77
Date:                Wed, 13 Dec 2023   Prob (F-statistic):          3.51e-162
Time:                        04:21:26   Log-Likelihood:                 29.082
No. Observations:                 569   AIC:                            -16.16
Df Residuals:                     548   BIC:                             75.06
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [22]:
print(models_bwd.loc[13, "model"].summary())

                            OLS Regression Results                            
Dep. Variable:              diagnosis   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     154.6
Date:                Wed, 13 Dec 2023   Prob (F-statistic):          3.04e-168
Time:                        04:22:49   Log-Likelihood:                 23.483
No. Observations:                 569   AIC:                            -20.97
Df Residuals:                     556   BIC:                             35.50
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     